In [8]:
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

load_dotenv()

True

In [9]:
llm = ChatOllama(model="deepseek-v3.1:671b-cloud",
                 base_url="https://ollama.com",
                 temperature=0.0)

embed_model = OllamaEmbeddings(model="qwen3-embedding:8b")

In [10]:
pdf_loader = PyPDFLoader("../ORDER_FLOW_Trading_Setups.pdf")

pdf_docs = pdf_loader.load()

doc0 = pdf_docs[0]
doc0

Document(metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '2021-01-05T16:05:37+01:00', 'moddate': '2021-01-05T16:05:37+01:00', 'source': 'ORDER_FLOW_Trading_Setups.pdf', 'total_pages': 161, 'page': 0, 'page_label': '1'}, page_content='')

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splitted_docs = text_splitter.split_documents(pdf_docs)
print(f"Было документов: {len(pdf_docs)}, стало фрагментов: {len(splitted_docs)}")

Было документов: 161, стало фрагментов: 237


In [16]:
vector = embed_model.embed_query("Пример текста для эмбеддинга")
print(len(vector), vector[:5])

4096 [0.0280245, 0.00563152, 0.010753281, -0.027322743, 0.00799139]


In [18]:
vector_store = FAISS.from_documents(splitted_docs, embed_model)

In [20]:
query = "trading setup?"
docs_found = vector_store.similarity_search(query, k=2)

for doc in docs_found:
    print(doc.metadata, doc.page_content[:50])

{'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '2021-01-05T16:05:37+01:00', 'moddate': '2021-01-05T16:05:37+01:00', 'source': 'ORDER_FLOW_Trading_Setups.pdf', 'total_pages': 161, 'page': 147, 'page_label': '148'} help. 
BONUS: As a special bonus to all of this, o
{'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '2021-01-05T16:05:37+01:00', 'moddate': '2021-01-05T16:05:37+01:00', 'source': 'ORDER_FLOW_Trading_Setups.pdf', 'total_pages': 161, 'page': 149, 'page_label': '150'} BONUS: Our specially trained tech support will do 


In [21]:
vector_store.save_local("faiss_index")

In [23]:
local_store = FAISS.load_local("faiss_index",
                               embed_model,
                               allow_dangerous_deserialization=True)

In [25]:
query = "trading setup?"
docs_found = local_store.similarity_search(query, k=2)

for doc in docs_found:
    print(doc.metadata, doc.page_content[:50])

{'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '2021-01-05T16:05:37+01:00', 'moddate': '2021-01-05T16:05:37+01:00', 'source': 'ORDER_FLOW_Trading_Setups.pdf', 'total_pages': 161, 'page': 147, 'page_label': '148'} help. 
BONUS: As a special bonus to all of this, o
{'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '2021-01-05T16:05:37+01:00', 'moddate': '2021-01-05T16:05:37+01:00', 'source': 'ORDER_FLOW_Trading_Setups.pdf', 'total_pages': 161, 'page': 149, 'page_label': '150'} BONUS: Our specially trained tech support will do 


---
### Использование Qdrant с LangChain
---

In [5]:
qdrant_client = QdrantClient("http://localhost:6333")

In [13]:
vector_store = QdrantVectorStore.from_documents(
    documents=splitted_docs,
    embedding=embed_model,
    url="http://localhost:6333",
    collection_name="order_flow_trading_setups_collection"
)

In [16]:
docs_found = vector_store.similarity_search("Volume Profile Setup", k=8)

for doc in docs_found:
    print(doc.page_content[:50])

Volume Profile Setup #2: Trend Setup .............
BONUS: Our specially trained tech support will do 
135 
 
Example #3: Volume Accumulation Setup 
 
 

b-Shaped Profile .................................
134 
 
EXAMPLES: Volume Accumulation Setup 
Exampl
How to Find Supports and Resistances with Volume P
from which you can later trade using the Order Flo
EXAMPLES: Volume Clusters (within a REJECTION) ...


In [17]:
retriever = vector_store.as_retriever(search_kwargs={"k": 8})
retriever

VectorStoreRetriever(tags=['QdrantVectorStore', 'OllamaEmbeddings'], vectorstore=<langchain_qdrant.qdrant.QdrantVectorStore object at 0x7fe8a7c04950>, search_kwargs={'k': 8})